<a href="https://colab.research.google.com/github/LucaMorocco/Luca-s-room/blob/main/Coding_School_data_science_project_Baseball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score

Functions and Libraries import

In [5]:
from google.colab import files
uploaded = files.upload()

#Base ball data set from caggle upload

Saving 500hits.csv to 500hits.csv


In [6]:
df_one = pd.read_csv('/content/500hits.csv',encoding = 'latin-1')
df_one.head()

,PLAYER,YRS,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,BA,HOF
0,Ty Cobb,24,3035,11434,2246,4189,724,295,117,726,1249,357,892,178,0.366,1
1,Stan Musial,22,3026,10972,1949,3630,725,177,475,1951,1599,696,78,31,0.331,1
2,Tris Speaker,22,2789,10195,1882,3514,792,222,117,724,1381,220,432,129,0.345,1
3,Derek Jeter,20,2747,11195,1923,3465,544,66,260,1311,1082,1840,358,97,0.310,1
4,Honus Wagner,21,2792,10430,1736,3430,640,252,101,0,963,327,722,15,0.329,1


In [7]:
df_one.info()
#It seems like there are no missing values,
#but there are probably incorrect values
# I saw a zero in RBIs, so definitly need to find zeros.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PLAYER  465 non-null    object 
 1   YRS     465 non-null    int64  
 2   G       465 non-null    int64  
 3   AB      465 non-null    int64  
 4   R       465 non-null    int64  
 5   H       465 non-null    int64  
 6   2B      465 non-null    int64  
 7   3B      465 non-null    int64  
 8   HR      465 non-null    int64  
 9   RBI     465 non-null    int64  
 10  BB      465 non-null    int64  
 11  SO      465 non-null    int64  
 12  SB      465 non-null    int64  
 13  CS      465 non-null    int64  
 14  BA      465 non-null    float64
 15  HOF     465 non-null    int64  
dtypes: float64(1), int64(14), object(1)
memory usage: 58.3+ KB


In [8]:
df_one.columns

Index(['PLAYER', 'YRS', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB',
       'SO', 'SB', 'CS', 'BA', 'HOF'],
      dtype='object')

In [11]:
rename={"PLAYER":"Player","YRS":"Years Played", "G":"Games Played",
        "AB":"At Bats","R":"Runs","H":"Hits","2B":"Doubles","3B":"Triples",
        "HR":"Home Runs","RBI":"Runs Batted In","BB":"Walks"
        ,"SO":"Strike Outs","SB":"Stolen Bases","CS":"Caught Stolen",
        "BA":"Batting Average","HOF":"Hall of Fame"}
df_one = df_one.rename(columns= rename)
df_one.head()

,Player,Years Played,Games Played,At Bats,Runs,Hits,Double,Triple,Home Run,Runs Batted In,Walks,Strike Outs,Stolen Bases,Caught Stolen,Batting Average,Hall of Fame
0,Ty Cobb,24,3035,11434,2246,4189,724,295,117,726,1249,357,892,178,0.366,1
1,Stan Musial,22,3026,10972,1949,3630,725,177,475,1951,1599,696,78,31,0.331,1
2,Tris Speaker,22,2789,10195,1882,3514,792,222,117,724,1381,220,432,129,0.345,1
3,Derek Jeter,20,2747,11195,1923,3465,544,66,260,1311,1082,1840,358,97,0.310,1
4,Honus Wagner,21,2792,10430,1736,3430,640,252,101,0,963,327,722,15,0.329,1
